# Ćwiczenie 6
*Tomasz Żebrowski*

In [64]:
import gym
import matplotlib.pyplot as plt
from IPython import display
import time
%matplotlib inline

env = gym.make("Taxi-v3", render_mode="ansi")
env.reset()
for i in range(20):
    display.clear_output(wait=True)
    print(env.render())
    env.step(env.action_space.sample())
    time.sleep(0.5)


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)

